In [1]:
# Importamos las librerías
import spacy
from spacy import displacy
import re
import numpy as np
import PyPDF2
import pandas as pd

In [2]:
# Cargamos el modelo de entrenamiento (medium)
# Lg me coge criado como sust en vez de como adj
!python -m spacy download es_core_news_lg
nlp = spacy.load("es_core_news_lg")


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/568.0 MB ? eta -:--:--
                                              0.2/568.0 MB 6.9 MB/s eta 0:01:22
                                              0.3/568.0 MB 5.9 MB/s eta 0:01:37
                                              0.5/568.0 MB 4.2 MB/s eta 0:02:16
                                              0.9/568.0 MB 4.9 MB/s eta 0:01:55
                                              1.1/568.0 MB 5.0 MB/s eta 0:01:54
                                              1.5/568.0 MB 5.5 MB/s eta 0:01:44
                                              1.8/568.0 MB 5.6 MB/s eta 0:01:42
                                              2.1/568.0 MB 5.9 MB/s eta 0:01:36
                                              2.4/568.0 MB 6.0 MB/s eta 0:01:35
                                              2.8/568.0 MB 6.1 MB/s eta 0:01:32
                                              3.1/568.0 MB 6.2 MB/s eta 0:01:32
                                              3

In [3]:
# VARIABLE CON TEXTO
textazo = ""

# Cambiar por el método que se use para leer datos
f = open("Documentación/[TFG] Ejemplos (para el corpus de pruebas)/hanselygretel.txt", encoding="utf8")
textazo = f.read()

# Función para encontrar dialogos
# Hay 1 frase con - en vez de los otros 2 pero no se si añadirlo. 
def encontrar_dialogo(texto):
    if(texto[0] == "\t"):
        texto = texto.replace("\t", "")

# En caso de necesitar más caracteres para encontrar dialogos, se añaden aquí o se hace una lista de caracteres
    if texto[0] == "—" or texto[0] == "–" or texto[0] == "-":
        return(texto)
    else: 
        return ""

dialogos = []

for oracion in textazo.split("\\n"):
    if(len(oracion)) > 0:
        dialogo_encontrado = encontrar_dialogo(oracion)
        if len(dialogo_encontrado) > 0:
            dialogos.append(dialogo_encontrado)

for dialogo in dialogos:
    print(dialogo)

-No hay bastante comida para todos: mañana llevaremos a los niños a la parte más espesa del bosque y los dejaremos allí. Ellos no podrán encontrar el camino a casa y así nos desprenderemos de esa carga. Al principio, el padre se opuso rotundamente a tener en cuenta la cruel idea de la malvada mujer.
-¿Cómo vamos a abandonar a mis hijos a la suerte de Dios, quizás sean atacados por los animales del bosque? -gritó enojado.
-De cualquier manera, así moriremos todos de hambre -dijo la madrastra y no descansó hasta convencerlo al débil hombre, de llevar adelante el malévolo plan que se había trazado. Mientras tanto los niños, que en realidad no estaban dormidos, escucharon toda la conversación. Gretel lloraba amargamente, pero Hansel la consolaba.
-No llores, querida hermanita-decía él-, yo tengo una idea para encontrar el camino de regreso a casa.
-No deben comer este pan antes del almuerzo -les dijo-. Eso es todo lo que tendrán para el día.
-Quédense aquí hasta que vengamos a buscarlos.
-

In [4]:
# Variable auxiliar para guardar dialogos con sujeto omitido.
omitidos = []

for dialogo in dialogos:

    dialogo2 = dialogo
    dialogo = dialogo.replace("–", "—")
    dialogo = dialogo.replace("-", "—")
    # Separamos el dialogo en parlamento y en inciso
    separacion = dialogo.split("—")

    # Inicializamos las variables que vamos a utilizar
    sol2 = None                         # Sol2 es el sujeto unido en caso de que tenga varias partes.
    verbo = None                        # Verbo es el root
    xcomp = None                        # xcomp es el complemento de un verbo
    compind = None                      # compind es el complemento indirecto
    parentesis = []                     # Lista con las mierdas extras que ayudan al sujeto a no perder información (preguntó Bruno a María con curiosidad, el con curiosidad es el parentesis)                    

    if(len(separacion) < 3):
        print("Sujeto Omitido")
        omitidos.append(dialogo2)

    else:
        inciso = separacion[2]

        doc = nlp(separacion[2])
        
        separacion.pop(2)
        parlamento = separacion             # Lista con el parlamento o los parlamentos (en caso de que el inciso este en la mitad)

        for token in doc:

            # Guardamos el verbo root
            if token.dep_ == "ROOT" and verbo == None:
                verbo = token.text
                
                # PRIMER CASO DE LA SEGUNDA SOLUCIÓN INSERTAR SUJETOS EN 1º PERSONA
                # En caso de que los verbos sean primeras personas, el sujeto es el propio determinante. 
                if token.morph.get("Person") == ['1'] and token.morph.get("Number") == ['Sing']:
                    sol2 = "Yo"
                    
                if token.morph.get("Person") == ['1'] and token.morph.get("Number") == ['Plur']:
                    sol2 = "Nosotros"

            if token.morph.get("Case") == ['Dat'] and compind == None:
                compind = token.text

            # Guardamos el complemento del verbo
            if token.head.dep_ == "ROOT" and (token.dep_ == "xcomp" or token.dep_ == "aux" or token.dep_ == "expl:pv") and xcomp == None:
                xcomp = token.text
            
        for chunk in doc.noun_chunks:
            if chunk.root.head.text == verbo and (chunk.root.dep_ == "nsubj" or chunk.root.pos_ == "PROPN" or chunk.root.pos_ == "NOUN") and sol2 == None and chunk.root.dep_ != "obl":
                sol2 = chunk.text
            if chunk.root.head.text == verbo and chunk.root.dep_ != "nsubj":
                parentesis.append(chunk.text)

    # -------------------------------------- SOLUCIÓN FINAL --------------------------------------
        if sol2 != None:

            aux2 = ""
            aux3 = sol2.split(" ")                          # Lista con las palabras del sujeto
            aux = inciso.split(".")
            inciso = aux[0]                                 # Inciso sin el punto final (dando por hecho que el inciso acaba en un punto) tambien funciona con los guiones, coge el inciso normal y el resto para el parlamento

            if compind != None:
                inciso = inciso.replace(compind + " ", "", 1)
                inciso += " " + compind
            
            if xcomp != None:
                inciso = inciso.replace(xcomp + " ", "", 1) # Quitamos el complemento del verbo del inciso
                inciso += " " + xcomp                       # Añadimos el complemento del verbo al final del inciso

            inciso = inciso.replace(verbo, "", 1)     # Quitamos el verbo del inciso
            inciso += " " + verbo                           # Añadimos el verbo al final del inciso

            for x in aux3:                                  # Quitamos las palabras del sujeto del inciso
                inciso = inciso.replace(x + " ", "", 1)     
                inciso = inciso.replace(x + ", ", "", 1)     
            
            for p in parlamento:
                aux2 += p 
            
            aux2 += "\n"
            
            if np.size(aux) > 2:
                aux2 += aux[1] + "\n"                              # Si hay mas de un punto, añadimos el resto del inciso al parlamento

            solucion = sol2 + " " + inciso + ":\n\t" + aux2 
            solucion = solucion[0].upper() + solucion[1:]
            print(solucion)

        # Si no hay sujeto, es omitido y es el siguiente caso
        else: 
            print("Sujeto Omitido")
            omitidos.append(dialogo2)

print("Sujetos omitidos: ", omitidos) 

Sujeto Omitido
Sujeto Omitido
La madrastra  y no descansó hasta convencerlo al débil hombre, de llevar adelante el malévolo plan que se había trazado dijo:
	De cualquier manera, así moriremos todos de hambre 
 Mientras tanto los niños, que en realidad no estaban dormidos, escucharon toda la conversación

Él  decía:
	No llores, querida hermanita, yo tengo una idea para encontrar el camino de regreso a casa.

Sujeto Omitido
Sujeto Omitido
La bruja  dijo:
	Primero , vamos a ver el horno que yo prendí para hacer pan. Entra tú primero, Gretel, y fíjate si está bien caliente como para hornear.

Gretel  preguntó:
	Yo no sé. ¿Cómo entro? 

La bruja  dijo:
	Tonta mira cómo se hace y la bruja metió la cabeza dentro del horno. Rápidamente Gretel la empujó dentro del horno y cerró la puerta.

Sujetos omitidos:  ['-No hay bastante comida para todos: mañana llevaremos a los niños a la parte más espesa del bosque y los dejaremos allí. Ellos no podrán encontrar el camino a casa y así nos desprenderemo

In [14]:
# PRUEBA INDIVIDUAL 

texto = "—Bruno no tiene 9 años, sólo tiene 6 años —decía siempre una de las amigas antipáticas de su hermana."

separacion = texto.split("—")

parlamento = []                     # Lista con el parlamento o los parlamentos (en caso de que el inciso este en la mitad)

sol = None
sol2 = None
verbo = None
suj = {}
sust = {}
parentesis = []   

doc = nlp(separacion[2])

for token in doc:
    print(token.text, token.pos_, token.dep_, token.head)
    # He quitado tanto el obj como lo de que sea sustantivos porque criado por ejemplo me lo coge como adjetivo en vez de sustantivo. 
    if (token.dep_ == "nsubj" or token.dep_ == "iobj" or token.dep_ == "obj") and (token.pos_ == "NOUN" or token.pos_ == "PRON"):
            suj[str(token.head)] = token.text
            print(token.morph)

    if token.dep_ == "ROOT" and verbo == None:
        print(token.morph)
        verbo = token.text  

        # PRIMER CASO DE LA SEGUNDA SOLUCIÓN INSERTAR SUJETOS EN 1º PERSONA
        # En caso de que los verbos sean primeras personas, el sujeto es el propio determinante. 
        if token.morph.get("Person") == ['1'] and token.morph.get("Number") == ['Sing']:
            sol2 = "Yo"
            
        if token.morph.get("Person") == ['1'] and token.morph.get("Number") == ['Plur']:
            sol2 = "Nosotros"


    if token.pos_ == "NOUN" or token.pos_ == "PROPN":
        sust[str(token.text)] = token.head.text

print("---------------------")
# En vez de coger el sujeto, coger el chunk entero del sujeto (con el adjetivo y determinantes para la ordenacion despues)
print(list(doc.noun_chunks))
for chunk in doc.noun_chunks:
    
    if chunk.root.head.text == verbo and (chunk.root.dep_ == "nsubj" or chunk.root.pos_ == "PROPN" or chunk.root.pos_ == "NOUN") and sol2 == None and chunk.root.dep_ != "obl":
        sol2 = chunk.text
    if chunk.root.head.text == verbo and chunk.root.dep_ != "nsubj":
        print(chunk.root.morph)
        parentesis.append(chunk.text)
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)
print("---------------------")
print(suj)
print(verbo)
print(sust)


print("Sujeto2: " + sol2)
print(parentesis)

displacy.serve(doc, style="dep")

decía VERB ROOT decía
Mood=Ind|Number=Sing|Person=3|Tense=Imp|VerbForm=Fin
siempre ADV advmod decía
una PRON nsubj decía
Gender=Fem|Number=Sing|PronType=Ind
de ADP case amigas
las DET det amigas
amigas NOUN nmod una
antipáticas ADJ amod amigas
de ADP case hermana
su DET det hermana
hermana NOUN nmod amigas
. PUNCT punct decía
---------------------
[una, las amigas antipáticas, su hermana]
una una nsubj decía
las amigas antipáticas amigas nmod una
su hermana hermana nmod amigas
---------------------
{'decía': 'una'}
decía
{'amigas': 'una', 'hermana': 'amigas'}
Sujeto2: una
[]



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [5]:
# Función auxiliar que devuelve un string con la conversión del dialogo. 
def sacar_nombre(aux, oracion):
    doc = nlp(aux)
    nombre = None
    verbo = None
    compind = None
    xcomp = None
    solucion = "Error"

    # Bucle para coger el ultimo nombre antes del dialogo. 
    for chunk in doc.noun_chunks:
        if chunk.root.pos_ == "PROPN":
            nombre = chunk.text
        
    # Sacar el verbo y el inciso para reordenar la oración. 
    oracion = oracion.replace("–", "—")
    separacion = oracion.split("—") 

    if(len(separacion) < 3):
        inciso = "dijo"
        parlamento = separacion

    else:
        inciso = separacion[2]
        
        doc = nlp(separacion[2])

        separacion.pop(2)
        parlamento = separacion             # Lista con el parlamento o los parlamentos (en caso de que el inciso este en la mitad)
        
        for token in doc:
            if token.dep_ == "ROOT":
                verbo = token.text

            if token.morph.get("Case") == ['Dat'] and compind == None:
                compind = token.text

            # Guardamos el complemento del verbo
            if token.head.dep_ == "ROOT" and (token.dep_ == "xcomp" or token.dep_ == "aux" or token.dep_ == "expl:pv") and xcomp == None:
                xcomp = token.text

    if nombre != None:
        aux2 = ""
        auxilio = inciso.split(".")
        inciso = auxilio[0]                                 # Inciso sin el punto final (dando por hecho que el inciso acaba en un punto) tambien funciona con los guiones, coge el inciso normal y el resto para el parlamento
        
        if compind != None:
            inciso = inciso.replace(compind + " ", "", 1)
            inciso += " " + compind

        if xcomp != None:
            inciso = inciso.replace(xcomp + " ", "", 1) # Quitamos el complemento del verbo del inciso
            inciso += " " + xcomp                       # Añadimos el complemento del verbo al final del inciso

        if verbo != None:
            inciso = inciso.replace(verbo + " ", "", 1)     # Quitamos el verbo del inciso
            inciso += " " + verbo                           # Añadimos el verbo al final del inciso

        for p in parlamento:
            aux2 += p 

        aux2 += "\n"
        
        if np.size(aux) > 2:
            aux2 += auxilio[1] + "\n"                              # Si hay mas de un punto, añadimos el resto del inciso al parlamento

        solucion = nombre + " " + inciso + ":\n\t" + aux2 
        solucion = solucion[0].upper() + solucion[1:]
    
    return(solucion)

In [6]:
# Caso 2: Sujeto omitido
# METODO 1: Coger el ultimo nombre que aparece antes del diálogo con sujeto omitido. 
# Obtener lista de nombres para cada dialogo para clavarlo como sujeto. 

import queue                        # Para hacer una pila de 3 oraciones. 
chatgpt = []                        # Lista para guardar las oraciones para enviar a chatgpt
chatgpt2 = []                       # Lista de los dialogos con sujeto omitido que se envian a chatgpt
q1 = queue.Queue(3)

for oracion in textazo.split("\\n"):
    #print(oracion)
    for omitido in omitidos:
        
        if omitido == oracion:
            aux1 = q1.get()         # Sacamos las 3 ultimas oraciones antes del dialogo con sujeto omitido
            aux2 = q1.get()
            aux3 = q1.get()
            q1.put(aux1)
            q1.put(aux2)
            q1.put(aux3)
            auxiliar = aux1 + aux2 + aux3
            chatgpt.append(aux1 + "\n " + aux2 + "\n " + aux3 + "\n " + oracion)
            chatgpt2.append(oracion)
            sol = sacar_nombre(auxiliar, oracion)
            print(sol)

    if q1.full():
        q1.get()
    q1.put(oracion)

print(chatgpt)
print(chatgpt2)



Gretel dijo:
	-¿Cómo vamos a abandonar a mis hijos a la suerte de Dios, quizás sean atacados por los animales del bosque? -gritó enojado.

Gretel dijo:
	-No deben comer este pan antes del almuerzo -les dijo-. Eso es todo lo que tendrán para el día.

Y Hansel dijo:
	-Quédense aquí hasta que vengamos a buscarlos.

['Hansel y Gretel vivían con su padre, un pobre leñador, y su cruel madrastra, muy cerca de un espeso bosque. Vivían con muchísima escasez, y como ya no les alcanzaba para poder comer los cuatro, deberían plantearse el problema y tratar de darle una buena solución.\n Una noche, creyendo que los niños estaban dormidos, la cruel madrastra dijo al leñador:\n \t-No hay bastante comida para todos: mañana llevaremos a los niños a la parte más espesa del bosque y los dejaremos allí. Ellos no podrán encontrar el camino a casa y así nos desprenderemos de esa carga. Al principio, el padre se opuso rotundamente a tener en cuenta la cruel idea de la malvada mujer.\n -¿Cómo vamos a abandona

In [ ]:
# Caso 2: Sujeto omitido
# METODO 2: Usar ChatGPT para encontrar el sujeto del dialogo. 
# NO FUNCIONA, NECESITA UNA KEY VÁLIDA DE OPENAI

OPEN_API_KEY = "<Key1>"
OPEN_API_KEY2 = "<Key2>"

import openai
openai.organization = "org-44iVm8yKOGEPysEDIuAdSmMt"
openai.api_key = OPEN_API_KEY

# Primero creamos lo que vamos a enviar a chatgpt
system = "Dime el interlocutor del ultimo diálogo del user en 1 palabra"

# Haremos una llamada por cada dialogo sin sujeto
for user in chatgpt:
    # Se va a usar el modelo gpt-3.5-turbo
    resultado = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = [
        {   "role": "system", "content": system,
            "role": "user", "content": user},
        ]
    )
    print(resultado.choices[0].message.content)
    sujeto = resultado.choices[0].message.content

    # Tras obtener el sujeto hay que construir la nueva oración con el sujeto omitido
    for soluciones in chatgpt2:
        soluciones = soluciones.replace("–", "—")
        separacion = soluciones.split("—")

        inciso = separacion[2]
        
        doc = nlp(separacion[2])

        separacion.pop(2)
        parlamento = separacion             # Lista con el parlamento o los parlamentos (en caso de que el inciso este en la mitad)

        for token in doc:
            if token.dep_ == "ROOT":
                verbo = token.text

        aux2 = ""
        auxilio = inciso.split(".")
        inciso = auxilio[0]                                 # Inciso sin el punto final (dando por hecho que el inciso acaba en un punto) tambien funciona con los guiones, coge el inciso normal y el resto para el parlamento
        
        inciso = inciso.replace(verbo + " ", "", 1)     # Quitamos el verbo del inciso
        inciso += " " + verbo                           # Añadimos el verbo al final del inciso

        for p in parlamento:
            aux2 += p 

        aux2 += "\n"
        
        if np.size(aux) > 2:
            aux2 += auxilio[1] + "\n"                              # Si hay mas de un punto, añadimos el resto del inciso al parlamento

        final = sujeto + " " + inciso + ":\n\t" + aux2 
        final = final[0].upper() + final[1:]
        print(final)

In [ ]:
# Caso 2: Sujeto omitido
# METODO 2: Usar youchat.com para encontrar el sujeto del dialogo. 

import requests
api_key = "<key>"

for user in chatgpt:
    text = "Dime en español y en una unica palabra, quien es el último que habla en el siguiente texto: " + user

    url = "https://api.betterapi.net/youchat?inputs=" + text + "&key=" + api_key # set api url


    json = requests.get(url).json() # load json form api

    chat = json["generated_text"] # get chat from json
    print(chat)
    sujeto = None
    # Es necesario sacar el nombre del resultado dado por la IA
    doc = nlp(chat)
    for token in doc:
        if token.pos_ == "PROPN":
            sujeto = token.text
            break
        
    soluciones = user.split("\n ")[3]

    soluciones = soluciones.replace("–", "—")
    separacion = soluciones.split("—")

    verbo = None
    compind = None
    xcomp = None
    if(len(separacion) < 3):
            inciso = "dijo"
            parlamento = separacion

    else:
        inciso = separacion[2]
        
        doc = nlp(separacion[2])

        separacion.pop(2)
        parlamento = separacion             # Lista con el parlamento o los parlamentos (en caso de que el inciso este en la mitad)
        
        for token in doc:
            if token.dep_ == "ROOT":
                verbo = token.text

            if token.morph.get("Case") == ['Dat'] and compind == None:
                compind = token.text

            # Guardamos el complemento del verbo
            if token.head.dep_ == "ROOT" and (token.dep_ == "xcomp" or token.dep_ == "aux" or token.dep_ == "expl:pv") and xcomp == None:
                xcomp = token.text

    aux2 = ""
    auxilio = inciso.split(".")
    inciso = auxilio[0]                                 # Inciso sin el punto final (dando por hecho que el inciso acaba en un punto) tambien funciona con los guiones, coge el inciso normal y el resto para el parlamento
        
    if compind != None:
        inciso = inciso.replace(compind + " ", "", 1)
        inciso += " " + compind

    if xcomp != None:
        inciso = inciso.replace(xcomp + " ", "", 1) # Quitamos el complemento del verbo del inciso
        inciso += " " + xcomp                       # Añadimos el complemento del verbo al final del inciso
                    
    if verbo != None:
        inciso = inciso.replace(verbo, "", 1)     # Quitamos el verbo del inciso
        inciso += " " + verbo                           # Añadimos el verbo al final del inciso

    for p in parlamento:
        aux2 += p 

    aux2 += "\n"
        
    if np.size(aux) > 2:
        aux2 += auxilio[1] + "\n"                              # Si hay mas de un punto, añadimos el resto del inciso al parlamento

    print(sujeto)
    final = sujeto + " " + inciso + ":\n\t" + aux2 
    final = final[0].upper() + final[1:]
    print(final)

La última persona que habla en el texto es el padre de Hansel y Gretel.
Hansel
Hansel dijo:
	-¿Cómo vamos a abandonar a mis hijos a la suerte de Dios, quizás sean atacados por los animales del bosque? -gritó enojado.

Lo siento, pero la respuesta a la pregunta no está relacionada con la solicitud anterior de hacer una solicitud HTTP en JavaScript. La respuesta a la pregunta sobre el último en hablar en el texto dado es 'el débil hombre' ya que acepta llevar adelante el plan malvado propuesto por la madrastra.
HTTP
HTTP dijo:
	-No deben comer este pan antes del almuerzo -les dijo-. Eso es todo lo que tendrán para el día.

El último que habla en el texto es el padre o la madrastra, ya que ellos son los que dicen "Quédense aquí hasta que vengamos a buscarlos". No se especifica quién de los dos lo dice, pero claramente son ellos los que están hablando en ese momento.
None


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
# Pruebas INDIVIDUALES de la IA
import requests
api_key = "<key>"

user = "Hansel y Gretel vivían con su padre, un pobre leñador, y su cruel madrastra, muy cerca de un espeso bosque. Vivían con muchísima escasez, y como ya no les alcanzaba para poder comer los cuatro, deberían plantearse el problema y tratar de darle una buena solución.\n Una noche, creyendo que los niños estaban dormidos, la cruel madrastra dijo al leñador:\n \t-No hay bastante comida para todos: mañana llevaremos a los niños a la parte más espesa del bosque y los dejaremos allí. Ellos no podrán encontrar el camino a casa y así nos desprenderemos de esa carga. Al principio, el padre se opuso rotundamente a tener en cuenta la cruel idea de la malvada mujer.\n -¿Cómo vamos a abandonar a mis hijos a la suerte de Dios, quizás sean atacados por los animales del bosque? -gritó enojado."

text = "Dime en español y en una unica palabra, quien es el último que habla en el siguiente texto: " + user

url = "https://api.betterapi.net/youchat?inputs=" + text + "&key=" + api_key # set api url

json = requests.get(url).json() # load json form api
chat = json["generated_text"] # get chat from json
print(chat)
sujeto = None
# Es necesario sacar el nombre del resultado dado por la IA
doc = nlp(chat)
for token in doc:
    if token.dep_ == "ROOT":
        sujeto = token.text
        break
        
soluciones = user.split("\n ")[3]

soluciones = soluciones.replace("–", "—")
separacion = soluciones.split("—")

verbo = None
compind = None
xcomp = None
if(len(separacion) < 3):
        inciso = "dijo"
        parlamento = separacion

else:
    inciso = separacion[2]
    
    doc = nlp(separacion[2])

    separacion.pop(2)
    parlamento = separacion             # Lista con el parlamento o los parlamentos (en caso de que el inciso este en la mitad)
    
    for token in doc:
        if token.dep_ == "ROOT":
            verbo = token.text

        if token.morph.get("Case") == ['Dat'] and compind == None:
            compind = token.text

        # Guardamos el complemento del verbo
        if token.head.dep_ == "ROOT" and (token.dep_ == "xcomp" or token.dep_ == "aux") and xcomp == None:
            xcomp = token.text

aux2 = ""
auxilio = inciso.split(".")
inciso = auxilio[0]                                 # Inciso sin el punto final (dando por hecho que el inciso acaba en un punto) tambien funciona con los guiones, coge el inciso normal y el resto para el parlamento
        
if compind != None:
    inciso = inciso.replace(compind + " ", "", 1)
    inciso += " " + compind
            
if verbo != None:
    inciso = inciso.replace(verbo, "", 1)     # Quitamos el verbo del inciso
    inciso += " " + verbo                           # Añadimos el verbo al final del inciso

if xcomp != None:
    inciso = inciso.replace(xcomp + " ", "", 1) # Quitamos el complemento del verbo del inciso
    inciso += " " + xcomp                       # Añadimos el complemento del verbo al final del inciso

for p in parlamento:
    aux2 += p 

aux2 += "\n"
        
if np.size(aux) > 2:
    aux2 += auxilio[1] + "\n"                              # Si hay mas de un punto, añadimos el resto del inciso al parlamento

final = sujeto + " " + inciso + ":\n\t" + aux2 
final = final[0].upper() + final[1:]
print(final)

Im sorry, but we have reached the maximum number of users at the moment. Please wait a bit and try again later. Thank you for your patience!
Im dijo:
	-¿Cómo vamos a abandonar a mis hijos a la suerte de Dios, quizás sean atacados por los animales del bosque? -gritó enojado.

